# Imports and Setup

In [1]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms, models
from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable
from scipy import ndimage
import pickle
import copy
import random
import time

torch.set_printoptions(precision=3)
cuda = True if torch.cuda.is_available() else False
torch.cuda.is_available()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Data Entry and Processing

In [2]:
# Transform image to tensor and normalize features from [0,255] to [0,1]
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5,),(0.5,),(0.5)),
                                ])

In [3]:
# Using CIFAR100
traindata = datasets.CIFAR100('./data', download=True, train=True, transform=transform)
testdata = datasets.CIFAR100('./data', download=True, train=False, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# Loaders that give 64 example batches
cifar_train_loader = torch.utils.data.DataLoader(traindata, batch_size=64, shuffle=True)
cifar_test_loader = torch.utils.data.DataLoader(testdata, batch_size=64, shuffle=True)

In [5]:
# Transform image to tensor and normalize features from [0,255] to [0,1]
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5,),(0.5,)),
                                ])

In [6]:
# Using MNIST
traindata = datasets.MNIST('./data', download=True, train=True, transform=transform)
testdata = datasets.MNIST('./data', download=True, train=False, transform=transform)

In [7]:
mnist_train_loader = torch.utils.data.DataLoader(traindata, batch_size=64, shuffle=True)
mnist_test_loader = torch.utils.data.DataLoader(testdata, batch_size=64, shuffle=True)


In [8]:
data, label = next(iter(mnist_train_loader))
print(data.size())
print(len(mnist_train_loader.dataset))
print(len(mnist_train_loader))
# for batch_idx, (data, target) in enumerate(mnist_train_loader):
#     print(batch_idx) #938
# print(mnist_train_loader)

torch.Size([64, 1, 28, 28])
60000
938


# Model

In [9]:
# Hyperparameters
log_interval = 10
num_classes = 100
torch.backends.cudnn.enabled = True
criterion = F.nll_loss

In [10]:
# load resnet 18 and change to fit problem dimensionality
resnet = models.resnet18()
resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False)
resnet.fc = nn.Sequential(nn.Linear(512, num_classes), nn.LogSoftmax(dim=1))
optimizer = optim.Adam(resnet.parameters())
# optimizer.to(device)
resnet = resnet.to(device)
print(resnet)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
# Training method that saves batch updates
def train(model, epoch, loader, returnable=False, amnesiac=True):
  model.train()
  deltas = []
  if amnesiac:
    for _ in range(50):
        delta = {}
        for param_tensor in model.state_dict():
          if "weight" in param_tensor or "bias" in param_tensor:
              delta[param_tensor] = 0
        deltas.append(delta)
    before = {}
    for param_tensor in model.state_dict():
        if "weight" in param_tensor or "bias" in param_tensor:
            before[param_tensor] = model.state_dict()[param_tensor].detach().cpu()
  for batch_idx, (data, target) in enumerate(loader):
    optimizer.zero_grad()
    output = model(data.to(device))
    loss = criterion(output, target.to(device))
    loss.backward()
    optimizer.step()
    if batch_idx % 10 == 0 and batch_idx < 500 and amnesiac:
      after = {}
      for param_tensor in model.state_dict():
        if "weight" in param_tensor or "bias" in param_tensor:
          after[param_tensor] = model.state_dict()[param_tensor].detach().cpu()
      for key in before:
        deltas[batch_idx // 10][key] = after[key] - before[key]
      for param_tensor in model.state_dict():
        if "weight" in param_tensor or "bias" in param_tensor:
          before[param_tensor] = model.state_dict()[param_tensor].detach().cpu()
    if batch_idx % log_interval == 0:
      print("\rEpoch: {} [{:6d}]\tLoss: {:.6f}".format(
          epoch, batch_idx*len(data),  loss.item()
      ), end="")
  return deltas

In [12]:
# Testing method
def test(model, loader, dname="Test set", printable=True):
  model.eval()
  test_loss = 0
  total = 0
  correct = 0
  with torch.no_grad():
    for data, target in loader:
      output = model(data.to(device))
      total += target.size()[0]
      test_loss += criterion(output, target.to(device)).item()
      _, pred = torch.topk(output, 1, dim=1, largest=True, sorted=True)
      for i, t in enumerate(target):
        if t in pred[i]:
            correct += 1
  test_loss /= len(loader.dataset)
  if printable:
    print('{}: Mean loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        dname, test_loss, correct, total, 
        100. * correct / total
        ))
  return 1. * correct / total

# Original Training

In [13]:
trainingepochs = 10
num_classes = 10
counter = 0
for param_tensor in resnet.state_dict():
    if "weight" in param_tensor or "bias" in param_tensor:
        counter+=1
print(counter)

62


In [14]:
# Train new model for n epochs, saving parameter updates for
# sensitive batches
deltas = []
for _ in range(50):
  delta = {}
  for param_tensor in resnet.state_dict():
    if "weight" in param_tensor or "bias" in param_tensor:
        delta[param_tensor] = 0
  deltas.append(delta)
for epoch in range(1, trainingepochs+1):
  starttime = time.process_time()
  # train(resnet, epoch, all_data_train_loader, returnable=False)
  batch = train(resnet, epoch, mnist_train_loader, returnable=True) 
  for i in range(50):
    for key in deltas[i]:
        deltas[i][key] = batch[i][key] + deltas[i][key]
  test(resnet, mnist_test_loader, dname="All data")
  print(f"Time taken: {time.process_time() - starttime}")

Epoch: 1 [ 59520]	Loss: 0.038096All data: Mean loss: 0.0013, Accuracy: 9750/10000 (98%)
Time taken: 21.827929384
Epoch: 2 [ 59520]	Loss: 0.207102All data: Mean loss: 0.0011, Accuracy: 9801/10000 (98%)
Time taken: 21.850851081000002
Epoch: 3 [ 59520]	Loss: 0.010645All data: Mean loss: 0.0007, Accuracy: 9872/10000 (99%)
Time taken: 21.676236935
Epoch: 4 [ 59520]	Loss: 0.012429All data: Mean loss: 0.0004, Accuracy: 9912/10000 (99%)
Time taken: 21.859814240000006
Epoch: 5 [ 59520]	Loss: 0.023939All data: Mean loss: 0.0004, Accuracy: 9910/10000 (99%)
Time taken: 21.753024909000004
Epoch: 6 [ 59520]	Loss: 0.044006All data: Mean loss: 0.0006, Accuracy: 9891/10000 (99%)
Time taken: 21.644823473999992
Epoch: 7 [ 59520]	Loss: 0.006646All data: Mean loss: 0.0007, Accuracy: 9867/10000 (99%)
Time taken: 21.788828181000014
Epoch: 8 [ 59520]	Loss: 0.035651All data: Mean loss: 0.0005, Accuracy: 9906/10000 (99%)
Time taken: 21.73796163
Epoch: 9 [ 59520]	Loss: 0.005040All data: Mean loss: 0.0004, Accura

In [15]:
with open('deltas.pickle', 'wb') as handle:
    pickle.dump(deltas, handle)

In [16]:
# # Train new model for n epochs, saving parameter updates for
# # ignoring 
# deltas = []
# for _ in range(50):
#   delta = {}
#   for param_tensor in resnet.state_dict():
#     if "weight" in param_tensor or "bias" in param_tensor:
#         delta[param_tensor] = 0
#   deltas.append(delta)
# for epoch in range(1, trainingepochs+1):
#   starttime = time.process_time()
#   # train(resnet, epoch, all_data_train_loader, returnable=False)
#   batch = train(resnet, epoch, mnist_train_loader, returnable=True) 
#   for i in range(50):
#     for key in deltas[i]:
#         deltas[i][key] = batch[i][key] + deltas[i][key]
#   test(resnet, mnist_test_loader, dname="All data")
#   print(f"Time taken: {time.process_time() - starttime}")

In [17]:
path = F"resnet/selective_mnist.pt"
torch.save({
            'model_state_dict': resnet.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, path)

In [18]:
path = F"resnet/selective_mnist.pt"
checkpoint = torch.load(path)
resnet.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [24]:
accuracy = [[],[],[],[],[],[],[],[],[],[]]
print(type(deltas))
# print(deltas)
# fileo = open(path, 'rb')
# deltas = pickle.load(fileo)
# fileo.close()
b = []
with open('deltas.pickle', 'rb') as handle:
    b = pickle.load(handle)
print(len(b))
print(len(deltas))
import os
print(os.path.getsize('deltas.pickle')/1073741824, ' GB')
# import sys
# print(sys.getsizeof(deltas)/1073741824)


<class 'list'>
50
50
2.0910925837233663  GB


In [19]:
# Begin amnesiac unlearning process, evaluating
# model accuracy as batches are removed

for j in range(10):
    random.shuffle(deltas)
    resnet.load_state_dict(checkpoint['model_state_dict'])
    for i in range(50):
        print(f"\riteration {j},{i}", end="")
        const = 1
        with torch.no_grad():
            state = resnet.state_dict()
            for param_tensor in state:
                if "weight" in param_tensor or "bias" in param_tensor:
                  state[param_tensor] = state[param_tensor] - const*deltas[i][param_tensor].cuda()
        resnet.load_state_dict(state)
        accuracy[j].append(test(resnet, mnist_test_loader, dname="All data"))

iteration 0,0All data: Mean loss: 0.0057, Accuracy: 9102/10000 (91%)
iteration 0,1All data: Mean loss: 0.0155, Accuracy: 8092/10000 (81%)
iteration 0,2All data: Mean loss: 0.0116, Accuracy: 7770/10000 (78%)
iteration 0,3All data: Mean loss: 0.0238, Accuracy: 7303/10000 (73%)
iteration 0,4All data: Mean loss: 0.0519, Accuracy: 5090/10000 (51%)
iteration 0,5All data: Mean loss: 0.0730, Accuracy: 4513/10000 (45%)
iteration 0,6All data: Mean loss: 0.0625, Accuracy: 4546/10000 (45%)
iteration 0,7All data: Mean loss: 0.0526, Accuracy: 5291/10000 (53%)
iteration 0,8All data: Mean loss: 0.0462, Accuracy: 4981/10000 (50%)
iteration 0,9All data: Mean loss: 0.0495, Accuracy: 4740/10000 (47%)
iteration 0,10All data: Mean loss: 0.0517, Accuracy: 4847/10000 (48%)
iteration 0,11All data: Mean loss: 0.0583, Accuracy: 4434/10000 (44%)
iteration 0,12All data: Mean loss: 0.0543, Accuracy: 4718/10000 (47%)
iteration 0,13All data: Mean loss: 0.0494, Accuracy: 4838/10000 (48%)
iteration 0,14All data: Mean l

In [20]:
path = F"selective_acc_mnist.pk"
with open(path, 'w') as f:
  for data in accuracy:
    f.write(f"{data},")

In [21]:
f = open(F"selective_acc_mnist.pk", "wb")
pickle.dump(accuracy, f)
f.close()

In [ ]:
f = open(F"selective_acc_mnist.pk", "rb")
pickle.load(accuracy, f)
f.close()